<a href="https://colab.research.google.com/github/joselopez11c/mi-primer-proyecto/blob/main/CARTAS_NOTARIALES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openpyxl
!pip install docxcompose
!pip install pydrive2

In [4]:
from google.colab import auth
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from oauth2client.client import GoogleCredentials

from docx import Document
from docxcompose.composer import Composer

import pandas as pd
import pickle
import os

In [60]:
# Autenticación
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Función para descargar y leer un archivo Excel de Google Drive
def read_excel_from_drive(file_id):
    downloadedExcel = drive.CreateFile({'id': file_id})
    downloadedExcel.FetchMetadata()
    print(f'Descargando: {downloadedExcel["title"]}')
    downloadedExcel.GetContentFile('temp.xlsx')
    df = pd.read_excel('temp.xlsx', parse_dates=['FECHA CARTA NOTARIAL'])
    return df

# Función para descargar la plantilla de Word de Google Drive
def download_word_template(file_id, filename):
    downloadedWord = drive.CreateFile({'id': file_id})
    downloadedWord.FetchMetadata()
    print(f'Descargando plantilla: {downloadedWord["title"]}')
    downloadedWord.GetContentFile(filename)

# Función para reemplazar las palabras clave en el documento
def replace_keywords_in_doc(doc, replacements):
    for para in doc.paragraphs:
        for run in para.runs:
            for key, value in replacements.items():
                if key in run.text:
                    run.text = run.text.replace(key, str(value))
    return doc


# Función para guardar el archivo en Google Drive
def save_to_drive(filename, folder_id):
    new_file = drive.CreateFile({'title': filename, 'parents': [{'id': folder_id}]})
    new_file.SetContentFile(filename)
    new_file.Upload()
    print(f'Archivo subido: {filename}')


def format_date(date):
    months = {
        1: "enero", 2: "febrero", 3: "marzo", 4: "abril",
        5: "mayo", 6: "junio", 7: "julio", 8: "agosto",
        9: "septiembre", 10: "octubre", 11: "noviembre", 12: "diciembre"
    }
    day = date.day
    month = months[date.month]
    year = date.year
    return f"{day} de {month} de {year}"


# Función para combinar dos documentos de Word sin perder formato
def append_docx(source, destination):
    composer = Composer(destination)
    composer.append(source)
    return destination

def clean_remove_guion(value):
    cleaned_value = value.strip()
    if cleaned_value.startswith('-'):
        cleaned_value = cleaned_value[1:].strip()  # Eliminar el guion al inicio y espacios en blanco
    return cleaned_value

# IDs de los archivos compartidos de Google Drive
file_id_excel = '10WyYPyLy3__kVQ7Vw79qZwG4gej0IxIz' # Base de datos para generar las cartas notariales
template_pc_id = '1SqavdNqkman2CKEnw-fZ0ZR2WuTI0tU-' # Plantilla de carta notarial para prestamo comercial
template_pc_fiadores_id = '1m_1CcabXddSUVzuZo2Z0d1XXJ0fc02bH' # Plantilla de fiadores para carta notarial por prestamo comercial
folder_id = '1-Nxg6P6uXLAA8IURYr_Sx1RQMP7uaapH' # Carpeta de salida para las cartas notariales

# Leer el archivo Excel en un DataFrame de pandas
df_excel = read_excel_from_drive(file_id_excel)


download_word_template(template_pc_id, 'template_pc.docx')
download_word_template(template_pc_fiadores_id, 'template_fiadores.docx')


df_excel

Descargando: BaseDatosParaCartasNotariales.xlsx
Descargando plantilla: PLANTILLA_CARTA_NOTARIAL_PC.docx
Descargando plantilla: PLANTILLA_CARTA_NOTARIAL_PC_FIADOR.docx


,TIPO CARTERA,CONTRATO,CLIENTE,DIRECCIÓN CLIENTE,DISTRITO CLIENTE,PROVINCIA CLIENTE,NRO DE DOCUMENTO,PRODUCTO,MONEDA,FECHA CARTA NOTARIAL,LIQUIDACIÓN,NOMBRE FIADORES,DIRECCIÓN FIADORES,DISTRITO FIADORES,PROVINCIA FIADORES
0,UNSECURED,1107129600096050,ORLANDO MEDRANO HUAMAN,- Av. 17 de Noviembre 5181\n- Av. 18 de Diciem...,- Comas\n- Los Olivos,- Huacho\n- Junin,21093440,P.Consumo,PEN,2024-05-07,212712.74,-Lesvia Medalit Chavez Aredo\n- Jose Ignacio L...,- Av. Las Castañas 366\n - Av. 28 de Julio 639...,- Ate\n- Miraflores,Chancay\nLima
1,UNSECURED,1103359600128131,M & R MULTISERVICIOSGENERALES Y TELECOMUNICACI...,Av. Quilca 15,SMP,Lima,20605627901,P.Comercial,PEN,2024-05-24,149858.20,- Juan Carlos Alvarado,Jr Abancay 051,Los Olivos,- Lima


In [61]:
# Función para procesar cada fila del DataFrame y generar documentos
def process_notarial_letter_dataframe(df_excel, folder_id):
    for index, row in df_excel.iterrows():
        process_row(index, row, folder_id)

# Función para procesar una fila individual
def process_row(index, row, folder_id):
    addresses = row['DIRECCIÓN CLIENTE'].split('\n')
    guarantors = row['NOMBRE FIADORES'].split('\n') if 'NOMBRE FIADORES' in row else []
    template_filename = 'template_pc.docx'
    template_fiadores_filename = 'template_fiadores.docx'
    # Crear un documento principal basado en la plantilla
    combined_doc = Document(template_filename)
    composer = Composer(combined_doc)

    # Procesar direcciones
    for addr_index, address in enumerate(addresses):
        process_address(addr_index, address, row, combined_doc, composer, template_filename)

    # Procesar fiadores
    for gua_index, guarantor in enumerate(guarantors):
        process_guarantor(gua_index, guarantor, row, combined_doc, composer, template_fiadores_filename)

    # Guardar el documento combinado
    filename = f'CARTA NOTARIAL RESOLUCIÓN CONTRATO {row["CONTRATO"]} - {row["CLIENTE"]}.docx'
    combined_doc.save(filename)
    save_to_drive(filename, folder_id)


# Función para procesar una dirección individual
def process_address(addr_index, address, row, combined_doc, composer, template_filename):
    cleaned_address = clean_remove_guion(address)

    replacements = {
        'P_FECHA': format_date(row['FECHA CARTA NOTARIAL']),
        'P_NOMBRE_CLIENTE': row['CLIENTE'],
        'P_DIRECCION_CLIENTE': cleaned_address,
        'P_DISTRITO_CLIENTE': clean_remove_guion(row['DISTRITO CLIENTE'].split('\n')[addr_index].strip()),
        'P_PROVINCIA_CLIENTE': clean_remove_guion(row['PROVINCIA CLIENTE'].split('\n')[addr_index].strip()),
        'P_NRO_CONTRATO': row['CONTRATO'],
        'P_TIPO_MONEDA': 'S/.' if row['MONEDA'] == 'PEN' else '$',
        'P_MONTO_LIQUIDACION': row['LIQUIDACIÓN'],
        'P_MONEDA': 'SOLES' if row['MONEDA'] == 'PEN' else 'DÓLARES',
    }

    if addr_index == 0:
        # Reemplazar palabras clave en el documento principal para la primera dirección
        replace_keywords_in_doc(combined_doc, replacements)
    else:
        # Crear una copia del documento de plantilla para direcciones adicionales
        doc_pc = Document(template_filename)
        replace_keywords_in_doc(doc_pc, replacements)

        # Añadir una sección de salto de página si no es la primera dirección
        doc_pc.add_page_break()

        # Combinar el documento modificado al documento principal
        composer.append(doc_pc)


# Función para procesar un fiador individual
def process_guarantor(gua_index, guarantor, row, combined_doc, composer, template_fiadores_filename):
    cleaned_guarantor = '\n'.join(part.strip() for part in guarantor.split('|'))
    replacements = {
        'P_FECHA': format_date(row['FECHA CARTA NOTARIAL']),
        'P_NOMBRE_CLIENTE': row['CLIENTE'],
        'P_NOMBRE_FIADOR': clean_remove_guion(cleaned_guarantor),
        'P_DIRECCION_FIADOR': clean_remove_guion(row['DIRECCIÓN FIADORES'].split('\n')[gua_index].strip()),
        'P_DISTRITO_FIADOR': clean_remove_guion(row['DISTRITO FIADORES'].split('\n')[gua_index].strip()),
        'P_PROVINCIA_FIADOR': clean_remove_guion(row['PROVINCIA FIADORES'].split('\n')[gua_index].strip()),
        'P_NRO_CONTRATO': row['CONTRATO'],
        'P_TIPO_MONEDA': 'S/.' if row['MONEDA'] == 'PEN' else '$',
        'P_MONTO_LIQUIDACION': row['LIQUIDACIÓN'],
        'P_MONEDA': 'SOLES' if row['MONEDA'] == 'PEN' else 'DÓLARES',
    }

    # Crear una copia del documento de plantilla de fiadores
    doc_fiador = Document(template_fiadores_filename)
    replace_keywords_in_doc(doc_fiador, replacements)

    # Añadir una sección de salto de página
    doc_fiador.add_page_break()

    # Combinar el documento modificado al documento principal
    composer.append(doc_fiador)

process_notarial_letter_dataframe(df_excel, folder_id)


Archivo subido: CARTA NOTARIAL RESOLUCIÓN CONTRATO 1107129600096050 - ORLANDO MEDRANO HUAMAN.docx
Archivo subido: CARTA NOTARIAL RESOLUCIÓN CONTRATO 1103359600128131 - M & R MULTISERVICIOSGENERALES Y TELECOMUNICACIONES SAC.docx
